## Loading Data


In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd

module_path = os.path.abspath(os.path.join(os.pardir, 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from modules import dataloading as dl

from datetime import datetime

targetdir = "../data/extracted/"

In [2]:
datetime.utcnow()

datetime.datetime(2020, 12, 15, 18, 52, 23, 553197)

In [3]:
# df = dl.SCF_load_data(targetdir, 2019, dl.sel_vars)
df = pd.read_stata('../data/extracted/scf2019s/p19i6.dta', columns=dl.sel_vars)
df.columns = [x.lower() for x in df.columns]
df.rename(columns=dl.rename_dict, inplace=True)
df.head()

,household_id,imputed_hh_id,weighting,persons_in_PEU,spouse_part_of_PEU,ref_next_relative_type,switch_of_resp_ref,ref_age,spouse_age,ref_sex,...,x3748,x3754,x3760,x3765,x3732,x3738,x3744,x3750,x3756,x3762
0,1,11,30598.896539,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
1,1,12,23561.874562,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
2,1,13,25726.122276,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
3,1,14,26488.317060,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
4,1,15,23809.061856,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df[df.household_id==2679].bonds_mkt_value

13310   NaN
13311   NaN
13312   NaN
13313   NaN
13314   NaN
Name: bonds_mkt_value, dtype: float64

In [8]:
df = dl.clean_SCF_df(df)

## Checking and cleaning cat vars

In [61]:
# Variables with negative values
negs_dict = {'dfs': {}, 'value_counts': {}}
for col in df:
    if df[col].min() < 0:
        negs_dict['dfs'][col] = df[df[col] < 0]
        negs_dict['value_counts'][col] = df[df[col] < 0][col].value_counts()
        
negs_dict.keys()
    

dict_keys(['dfs', 'value_counts'])

These are all of the variables that seem to have negative values, which should not be the case. Investigating case-by-case below.

### `ref_race`

In [62]:
var = 'ref_race'
negs_dict['value_counts'][var]

-7.0    324
-0.6      7
-2.2      1
-0.4      1
-1.8      1
-2.0      1
Name: ref_race, dtype: int64

In [63]:
df[df[var] == -7.0].index

Int64Index([  20,   25,   33,   59,   69,   78,   97,  151,  168,  171,
            ...
            5615, 5631, 5643, 5654, 5683, 5715, 5737, 5755, 5778, 5811],
           dtype='int64', name='household_id', length=324)

In [64]:
og_df = pd.read_stata('../data/extracted/scf2019s/p19i6.dta', columns=dl.sel_vars)
og_df.columns = [x.lower() for x in og_df.columns]
og_df.rename(columns=dl.rename_dict, inplace=True)
og_df.head()

,household_id,imputed_hh_id,weighting,persons_in_PEU,spouse_part_of_PEU,ref_next_relative_type,switch_of_resp_ref,ref_age,spouse_age,ref_sex,...,x3748,x3754,x3760,x3765,x3732,x3738,x3744,x3750,x3756,x3762
0,1,11,30598.896539,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
1,1,12,23561.874562,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
2,1,13,25726.122276,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
3,1,14,26488.317060,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0
4,1,15,23809.061856,1,1,0,5,75,0,2,...,0,0,0,0,0,0,0,0,0,0


In [65]:
og_df[og_df.household_id == 20][var]

95   -7
96   -7
97   -7
98   -7
99   -7
Name: ref_race, dtype: int8

-7 means other, so all of these households can go to 0. 


In [66]:
df[df[var] == -0.6].index

Int64Index([226, 233, 669, 2176, 2286, 3617, 3903], dtype='int64', name='household_id')

In [67]:
og_df[og_df.household_id == 226][var]

1120    1
1121   -7
1122    1
1123    1
1124    1
Name: ref_race, dtype: int8

In [68]:
og_df[og_df.household_id == 233][var]

1155    1
1156    1
1157    1
1158    1
1159   -7
Name: ref_race, dtype: int8

For some reason the imputed dataset also seems to impute race to other in some cases. Anything that is not -7 for race may have to go to 1.

In [69]:
df[df[var] == -1.8].index

Int64Index([46], dtype='int64', name='household_id')

In [70]:
og_df[og_df.household_id == 46][var]

225   -7
226    2
227    1
228   -7
229    2
Name: ref_race, dtype: int8

I think given the mixture of races in the imputed data for each HH and their relatively low number, all of these will go to 0.

### `ref_educ`

In [53]:
var = 'ref_educ'
negs_dict['value_counts'][var]

-1.0    9
Name: ref_educ, dtype: int64

-1 means less than first grade so these will go to 0.

### `spouse_educ`

In [55]:
var = 'spouse_educ'
negs_dict['value_counts'][var]

-1.0    6
Name: spouse_educ, dtype: int64

Same as `ref_educ`

### `total_income`

In [71]:
var = 'total_income'
negs_dict['value_counts'][var]

-9.0    36
-1.0     9
Name: total_income, dtype: int64

-1 is nothing and -9 is negative income generally, so all of these will go to 0. 

### `life_ins_cash_value`

In [72]:
var = 'life_ins_cash_value'
negs_dict['value_counts'][var]

-1.0    42
Name: life_ins_cash_value, dtype: int64

-1 is also nothing, so goes to 0. This is getting tedious. I've adjusted the dict.

In [73]:
negs_dict['value_counts']

{'ref_race': -7.0    324
 -0.6      7
 -2.2      1
 -0.4      1
 -1.8      1
 -2.0      1
 Name: ref_race, dtype: int64, 'ref_educ': -1.0    9
 Name: ref_educ, dtype: int64, 'spouse_educ': -1.0    6
 Name: spouse_educ, dtype: int64, 'total_income': -9.0    36
 -1.0     9
 Name: total_income, dtype: int64, 'life_ins_cash_value': -1.0    42
 Name: life_ins_cash_value, dtype: int64, 'total_cc_limit': -1.0    12
 Name: total_cc_limit, dtype: int64, 'num_fin_inst': -1.0    136
 -0.6      1
 Name: num_fin_inst, dtype: int64, 'cc_newcharges_value': -1.0    290
 -2.0    169
 -3.0      8
 Name: cc_newcharges_value, dtype: int64, 'cc_currbal_value': -2.0    1219
 -1.0    1071
 -3.0     259
 -1.6       1
 -2.2       1
 -2.8       1
 -2.6       1
 -2.4       1
 Name: cc_currbal_value, dtype: int64, 'checking_accts_value': -1.0    53
 -2.0     5
 Name: checking_accts_value, dtype: int64, 'savings_accts_value': -1.0    8
 -2.0    3
 -5.0    1
 Name: savings_accts_value, dtype: int64, 'lqd_assets': -

## Saving a Sample

In [ ]:
csv_head = df.head()

In [ ]:
csv_head.to_csv('example_data.csv', index=False)

In [ ]:
csv_head.shape

## Checking zips

In [ ]:
url = 'https://www.federalreserve.gov/econres/files/scf2019s.zip'

year = 2019
targetzip = targetdir + f'SCF{str(year)}_data_public.zip'


output = dl.URL_DL_ZIP(targetzip, targetdir, url)
output

In [ ]:
# Add Implicate Number
df['implicate'] = [x - y*10 for x, y in zip(df['imputed_hh_id'], df['household_id'])]
# weighting dividing by 5 since data implicates being combined for regression
df['across_imp_weighting'] = [x/5 for x in df['weighting']]

In [ ]:
df.head()

In [ ]:
targetzip = 'data/extracted/target.zip'
targetzip.rsplit('.', 1)[0]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,8))
sns.set_theme(style="darkgrid")

sns.scatterplot(
    data = df[df.implicate ==1], y="lqd_net_worth", x="total_income",
)

## IID Random Sampling

Consider how to do that with this dataset

## RII Regression

In [ ]:
dl.RII(df,
       Xseries=['stock_mkt_value'],
       y='total_income')

In [ ]:
output[1]

## Archiving and Logging

In [ ]:
url = 'https://www.federalreserve.gov/econres/files/scf2016s.zip'

df = dl.archive(url = url,
          targetdir = targetdir)

df.index.max()



In [ ]:
df

In [ ]:
d = df.to_dict()

for key in d:
    print(key)
    for subkey in d[key]:
        print(subkey)
        print(type(d[key][subkey]))

## Misc.

In [ ]:
len(coeffs[i])

In [ ]:
import numpy as np
from scipy import stats

coeffs = output[0]
m = 5
s = []
k = len(coeffs[1][0])

#  point estimates for each coeff 

for n in range(k):
    s.append([])


for i in range(m):
    i += 1
    for n in range(k):
        s[n].append(coeffs[i][0][n])
        
Qm_bar = []    
for n in range(k):
    
    ssum = sum(s[n]) / m
    Qm_bar.append(ssum)
    
Qm_bar = np.array([Qm_bar])    


    

# var-cov matrix of point estimates
summand_set = np.zeros((k, k))
for i in range(m):
    i+=1
    var = coeffs[i] - Qm_bar

    summand = var.T * var
    
    summand_set = summand_set + summand
    
Bm = summand_set / (m-1)


# avg of variance-cov matrices
summand_set = np.zeros((k,k))
var_matrices = output[1]

for i in range(m):
    i+=1
    summand = var_matrices[i]
    
    summand_set = summand_set + summand
    
Um_bar = summand_set/m


# total variance of regression coeff.
Tm = Um_bar + (1 + m**(-1))*Bm


# std dev of regression coeff.
Stddev = Tm**(1/2)

# t stats of regression coeff.
t_stats = Qm_bar/Stddev

    
# Relative increase in variance due to nonresponse
R_m =  ((1 + m**(-1))*Bm
      / Um_bar)

# Degrees of freedom
v = ((m-1)
     *(1+R_m**(-1))**(2))

p_values = []
for i in range(k):
    p_values.append(stats.t.sf(abs(t_stats[i][i]), df=v[i][i])*2) 
    
# P-values    
p_dict = {}
X_vars = ['intercept','ref_age','doctorate_deg', 'professional_deg', 'master_deg', 'college_deg', 'hs_deg']
for i in range(k):
    p_dict[X_vars[i]] = p_values[i] 
p_dict    

In [ ]:
df.head()

In [ ]:
weights = ((df.weighting.sum())/5)

n = (df.shape[0]/5)

print(f'weights: {weights}')
print(f'n: {n}')
print(f'n: {weights*n}')

In [ ]:
list(df.keys())[3:-3]

In [ ]:
# financially independent between 21 and 35
query = (df['ref_age'] <= 35) & (df['persons_in_PEU'] == 1) & (df['ref_age'] >= 21)

len(df[query].household_id.unique())
df[query].implicate.value_counts()

In [ ]:
import matplotlib.pyplot as plt

#financially independent between 21 and 30

query = (df['x8022'] <= 30) & (df['x7001'] == 1) & (df['x8022'] >= 21)
x = df[query].x8022.value_counts().index
y = df[query].x8022.value_counts().values

plt.bar(x=x,
       height=y)

plt.show()

## number of people in PEUs


In [ ]:
df = dl.clean_df(df)
df.sample(5)

In [ ]:
"""XX1             Internal case ID (8-digit) -- identical for
                all imputation replicates ("implicates")
                
                NOTE: includes IDs of cases not released to the field (list
                sample replicate cases and postcard refusals)
                *********************************************************
                    NOT INCLUDED IN THE PUBLIC DATA SET
                *********************************************************


X1              Internal ID/replicate number -- XX1*10 + imputation replicate
                number (1-5)
                *********************************************************
                    NOT INCLUDED IN THE PUBLIC DATA SET
                *********************************************************

YY1             Case ID for external users

                     0.     Inap. (not a completed interview)

Y1              Equivalent for X1 for external users

                     0.     Inap. (not a completed interview)
                     
Regional data not included in public dataset 

"""

"""EDUCATION
 1.    *1st, 2nd, 3rd, or 4th grade
                     2.    *5th or 6th grade
                     3.    *7th and 8th grade
                     4.    *9th grade
                     5.    *10th grade
                     6.    *11th grade
                     7.    *12th grade, no diploma
                     8.    *High school graduate - high school diploma or equivalent
                     9.    *Some college but no degree
                    10.    *Associate degree in college - occupation/vocation program
                    11.    *Associate degree in college - academic program
                    12.    *Bachelor's degree (for example: BA, AB, BS)
                    13.    *Master's degree ( for exmaple: MA, MS, MENG, MED, MSW, MBA)
                    14.    *Professional school degree (for example: MD, DDS, DVM, LLB, JD)
                    15.    *Doctorate degree (for example: PHD, EDD)
                    -1.    *Less than 1st grade
                     0.     Inap. (no spouse/partner;)
                *********************************************************
                    FOR THE PUBLIC DATA SET, CODES 2, 3, 4, 5, 6, AND 7
                    ARE COMBINED WITH CODE 1; CODE 10 AND CODE 11 ARE
                    COMBINED WITH CODE 9, AND; CODES 13, 14, AND 15 ARE
                    COMBINED WITH CODE 12
                *********************************************************

"""
rename_dict = {'yy1': 'household_id',
               'y1': 'imputed_hh_id',
               'x42001': 'weighting',
               'x7001': 'persons_in_PEU',
               'x7020': 'spouse_part_of_PEU', ## 1 is not in PEU, 2 is in
               'x102': 'ref_next_relative_type', ## 2 and 3 are spouses/partners, 1 is the respondent 
               'x8000': 'switch_of_resp_ref',
               'x14': 'ref_age',
               'x19': 'spouse_age',
               'x8021': 'ref_sex',
               'x103': 'spouse_sex',
               'x6809': 'ref_race',
               'x6810': 'spouse_race',
               'x5931': 'ref_educ', ## doctorate and profession combined as code 14
               'x6111': 'spouse_educ',
               'x6780': 'ref_UE_last_year', # UE or looking for work in last year
               'x6784': 'ref_UE_last_year',
               'x7402': 'ref_industry_code', # 1 is agr, 2 energy/construction, 3 food, 4 trade/retail, 5 prof. services, 6 consumer services/goods, 7 public admin
               'x7412': 'spouse_industry_code',
               'x7401': 'ref_occ_code', # 1 is white-collar/services, 2 is front-line clerks/sales, 3 public institutions/entertainemnt front-line, 4 artisans, 5 industrial workers, 6 industrial managers
               'x7411': 'spouse_occ_code',
               'x501': 'primary_home_type', # 2 is mobile home, 2 is house/apt, 3 ranch, 5 farm
               'x7136': 'chance_staying_home', # 0 - 100
                   'x6026': 'ref_mom_living',
                   'x6120': 'spouse_mom_living',
                   'x6032': 'ref_mom_educ',
                   'x6132': 'spouse_mom_educ',
                   'x6027': 'ref_mom_age', ## FOR THE PUBLIC DATA SET, PARENTS' AGES ROUNDED TO NEAREST AND TOP-CODED AT 95
                   'x6121': 'spouse_mom_age',
                   'x6028': 'ref_dad_living',
                   'x6122': 'spouse_dad_living',
                   'x6033': 'ref_dad_educ',
                   'x6133': 'spouse_dad_educ',
                   'x6029': 'ref_dad_age',
                   'x6123': 'spouse_dad_age',
              'x5729': '2018_total_income',
              'x7650': 'income_comparison', # 1 is high, 2 is low, 3 is normal compared to normal year
              'x5802': 'inheritances',
              'x6704': 'mutual_funds_value',
              'x6706': 'bonds_mkt_value',
              'x3915': 'stock_mkt_value',
              'x6576': 'annuity_cash_value',
              'x6587': 'trusts_cash_value',
              'x4006': 'life_ins_cash_value',
              'x414': 'total_cc_limit',
              'x432': 'freq_cc_payment',
              'x7575': 'rev_charge_accts'}

df.rename(columns=rename_dict, inplace=True)
df.head()

In [ ]:


## Lines of credit
LOC_owed_list = ['x1108',
                 'x1119',
                 'x1130',
                 'x1136'
                ]

df['LOC_owed_now'] = (df['x1108']
                    + df['x1119']
                    + df['x1130']
                    + df['x1136']
                   )

## Education loans
educ_loans_owed_list = ['x7824',
                        'x7847',
                        'x7870',
                        'x7924',
                        'x7947',
                        'x7970',
                        'x7179'
                       ]

df['ed_loans_owed_now'] = (df['x7824']
                           + df['x7847']
                           + df['x7870']
                           + df['x7924']
                           + df['x7947']
                           + df['x7970']
                           + df['x7179']
                          )



## Relatives living in HH
"""
                     1.    *RESPONDENT
                     2.    *SPOUSE; Spouse of R
                     3.    *PARTNER; Partner of R
                     4.    *CHILD (in-law) (of R or Spouse/Partner)
                     5.    *GRANDCHILD
                     6.    *PARENT
                     7.    *GRANDPARENT
                     8.    *AUNT/UNCLE
                     9.    *COUSIN
                    10.    *NIECE/NEPHEW
                    11.    *SISTER/BROTHER
                    12.    *GREAT GRANDCHILD
                    29.    *OTHER RELATIVE
                    31.    *ROOMMATE
                    32.    *FRIEND
                    34.    *BOARDER OR ROOMER/LODGER
                    35.    *PAID HELP; maid, etc.
                    36.    *FOSTER CHILD
                    39.    *OTHER UNRELATED PERSON
                     0.     Inap. (no further persons)
                *********************************************************
                    FOR THE PUBLIC DATA SET, CODE 12 IS COMBINED WITH
                    CODE 5; CODES 31, 32, AND 36 ARE COMBINED WITH CODE
                    39; CODES 9 AND 10 ARE COMBINED WITH CODE 29
                *********************************************************
"""

person_types_in_HH = ['x8020', 
                      'x102',
                      'x108',
                      'x114',
                      'x120',
                      'x126',
                      'x132',
                      'x202',
                      'x208',
                      'x214',
                      'x220',
                      'x226'
                     ]

## CREDIT cards
cc_newcharges_list = ['x412',
                 'x420',
                 'x426'
                ]

df['cc_newcharges_value'] = (df['x412']
                     + df['x420']
                     + df['x426']
                    )

cc_currbal_list = ['x413',
                 'x421',
                 'x427'
                ]

df['cc_currbal_value'] = (df['x413']
                     + df['x421']
                     + df['x427']
                    )

## CHECKING Nos. 1-6 have detailed data, 7 is remaining accounts

checking_accts_list = ['x3506',
                       'x3510',
                       'x3514',
                       'x3518',
                       'x3522',
                       'x3526',
                       'x3529'
                      ]

df['checking_accts_value'] = (df['x3506']
                              + df['x3510']
                              + df['x3514']
                              + df['x3518']
                             + df['x3522']
                             + df['x3526']
                             + df['x3529']
                             )

## SAVINGS accts

"""

                     1.    *TRADITIONAL SAVINGS ACCOUNT; "passbook account";
                            "statement account"
                     2.    *COVERDELL/EDUCATION IRA
                     3.    *529/STATE-SPONSORED EDUCATION ACCOUNT
                     4.    *MONEY MARKET ACCOUNT
                     5.     Christmas club account; other account for
                            designated saving purpose (e.g., vacation)
                     6.     Share account
                     7.    *HEALTH SAVINGS ACCOUNT; medical savings account
                    12.    *OTHER FLOATING-RATE SAVINGS ACCOUNT
                            (other than those coded 4)
                    14.     Informal group saving arrangement
                    20.     Foreign account type
                    30.    *SWEEP ACCOUNT n.e.c.; cash management account
                    -7.    *OTHER
                     0.     Inap. (no savings accounts: X3727^=1/fewer than 2
                            accounts: X3728<2/fewer than 3 account: X3728<3/
                            fewer than 4 accounts: X3728<4/fewer than 5
                            accounts: X3728<5/fewer than 6 accounts)
                *********************************************************
                    FOR THE PUBLIC DATA SET, CODES 6, 14, AND 20 ARE
                    COMBINED WITH CODE 1; CODES 3 AND 7 ARE COMBINED
                    WITH CODE 2; CODE 30 IS COMBINED WITH CODE 12
                *********************************************************
"""
## Nos. 1-6 have detailed data, 7 is remaining accounts
savings_accts_list = ['x3730',
                      'x3736',
                      'x3742',
                      'x3748',
                      'x3754',
                      'x3760',
                      'x3765'
                     ]

savings_accts_types = [df['x3732'],
                          df['x3738'],
                          df['x3744'],
                          df['x3750'],
                          df['x3756'],
                          df['x3762']
                      ]

savings_accts_incl_codes = [1, 4, 12]

# inlcuding only unincumbered savings
for i in savings_accts_list:
    for n in savings_accts_types:
        df[i] = [(y if x in savings_accts_incl_codes 
                  else 0) 
                 for x, y in zip(n, df[i])]

df['savings_accts_value'] = (df['x3730']
                              + df['x3736']
                              + df['x3742']
                              + df['x3748']
                             + df['x3754']
                             + df['x3760']
                             + df['x3765']
                            )
                      